In [1]:
import os
import cv2
import sys
from scipy import io
import pandas as pd
from scipy.spatial import distance
import numpy as np
import math
import matplotlib.pyplot as plt
import glob
import mecha.fracture

def make_mask(path):
    class PolygonDrawer(object):
        def __init__(self, window_name):
            self.window_name = window_name # Name for our window
            self.done = False # Flag signalling we're done
            self.current = (0, 0) # Current position, so we can draw the line-in-progress
            self.points = [] # List of points defining our polygon

        def on_mouse(self, event, x, y, buttons, user_param):
            # Mouse callback that gets called for every mouse event (i.e. moving, clicking, etc.)
            if self.done: # Nothing more to do
                return
            if event == cv2.EVENT_MOUSEMOVE:
                # We want to be able to draw the line-in-progress, so update current mouse position
                self.current = (x, y)
            elif event == cv2.EVENT_LBUTTONDOWN:
                # Left click means adding a point at current position to the list of points
                # print("Adding point #%d with position(%d,%d)" % (len(self.points), x, y))
                self.points.append((x, y))
            elif event == cv2.EVENT_RBUTTONDOWN:
                # Right click means we're done
                print("Completing polygon with %d points." % len(self.points))
                self.done = True

        def run(self):
            # Let's create our working window and set a mouse callback to handle events
            cv2.namedWindow(self.window_name)
            cv2.imshow(self.window_name, np.zeros(CANVAS_SIZE, np.uint8))
            cv2.waitKey(1)
            cv2.setMouseCallback(self.window_name, self.on_mouse)

            while(not self.done):
                # This is our drawing loop, we just continuously draw new images
                # and show them in the named window
                canvas = images[i]
                if (len(self.points) > 0):
                    # Draw all the current polygon segments
                    cv2.polylines(canvas, np.array([self.points]), False, (255, 255, 255), 1)
                    # And  also show what the current segment would look like
                    # cv2.line(canvas, self.points[-1], self.current, WORKING_LINE_COLOR)
                # Update the window
                cv2.imshow(self.window_name, canvas)
                # And wait 50ms before next iteration (this will pump window messages meanwhile)
                if cv2.waitKey(50) == 27: # ESC hit
                    self.done = True
            # User finised entering the polygon points, so let's make the final drawing
            canvas = images[i]
            # of a filled polygon
            if (len(self.points) > 0):
                cv2.fillPoly(canvas, np.array([self.points]), (255, 255, 255))
                # print(np.array([self.points]))
            # And show it
            cv2.imshow(self.window_name, canvas)
            # Waiting for the user to press any key
            cv2.waitKey()
            cv2.destroyWindow(self.window_name)
            return canvas, self.points

    # file_list = os.listdir(path)
    # img = [file for file in file_list if file.endswith(".jpg")]
    # image = cv2.imread(path+'/'+img[num], cv2.IMREAD_GRAYSCALE)
    images = load_file(path,flip='')

    # if image is None:
    #     print('Image load failed!')
    #     sys.exit()
    for i in images:
        x, y, w, h = cv2.selectROI(images[i])
        cv2.destroyAllWindows()

        cv2.rectangle(images[i], (x,y), (x+w,y+h),(0,0,0),2)
        CANVAS_SIZE = np.shape(images[i])

        poly = PolygonDrawer("Polygon")
        _, poly_points = poly.run()

        ##mask part
        mask = np.zeros(np.shape(images[i]))
        mask[y:y+h, x:x+w] = 255
        cv2.fillPoly(mask, np.array([poly_points]), (0,0,0))
        cv2.imshow('mask', mask)

        #%% save part
        cv2.imwrite(path+'/'+str(i)+'_mask.png',mask)
        cv2.waitKey()
        cv2.destroyAllWindows()
    return

class Crack_tip():
    def __init__(self, ratio, key, initial_crack, thickness, width):
        """ x, y 는 좌표, point 는 리스트로 x,y좌표 모아줌, count는 좌표 몇개, 길이, 1픽셀당 mm(mm/1픽셀)"""
        self.B = thickness
        self.W = width
        self.x = 0
        self.y =0
        self.inital_crack = initial_crack
        self.point = []
        self.count= -1
        self.length = 0
        self.ratio = ratio
        self.df = pd.DataFrame(index = [i for i in key], columns=['length(mm)',  'length(add)(mm)', 'x', 'y', 'Load(N)','ASTM_E399(K)'])

        self.key= list(key)
        return

    def __inital__(self, x, y, load):
        self.x = x
        self.y= y
        self.point.append([x,y])
        self.df.loc[self.key[0]] = [self.inital_crack ,self.inital_crack , x, y,
                                    load, ASTM_E399(load*0.001,self.inital_crack*0.1,self.B*0.1,self.W*0.1)]

        self.count +=1


    def append(self, x, y, load):
        if self.count == -1:
            self.__inital__(x,y,load)
        else:
            self.x =x
            self.y= y
            self.point.append([x,y])
            self.count +=1

            self.length = distance.cdist([self.point[self.count-1]], [self.point[self.count]]) * self.ratio
            self.df.loc[self.key[self.count]] = [ self.length[0][0] ,self.length[0][0] + self.df["length(add)(mm)"][self.count-1], x, y,
                                                  load, ASTM_E399(load*0.001, (self.length + self.df["length(add)(mm)"][self.count-1])[0][0]*0.1,self.B*0.1,self.W*0.1)]
            print(type((self.length + self.df["length(add)(mm)"][self.count-1])[0][0]))
    def position(self):
        return self.point

    def all_position(self):
        return self.df
    pass


def ASTM_E399(Pq,a,B,W):
    """B is thickness(cm), W is width(cm), Pq is stress(kN), A is crack length(cm)"""
    f_in = a/W
    f=((2+f_in)*(0.886+4.64*f_in-13.32*(f_in**2)+14.72*(f_in**3)-5.6*(f_in**4)))/((1-f_in)**1.5)
    kq=(Pq*f)/(B*(W**0.5))
    return kq

def hookes_law_sigmayy(YoungsModulus, Poisson, strain_xx, strain_yy):
    sigma_yy = YoungsModulus*(strain_xx*Poisson + strain_yy)/(1-Poisson**2)
    return sigma_yy

def Williams_sigmayy(r, theta):
    theta = math.radians(theta)
    K1_coefficient = np.cos(theta/2) * (1 + np.sin(theta/2) * np.sin(3 * theta / 2)) / (2 * np.pi * r) ** 0.5
    K2_coefficient = np.sin(theta/2) * np.cos(theta/2) * np.cos(3*theta/2) / (2 * np.pi * r) ** 0.5
    return [K1_coefficient, K2_coefficient]

def cart2pol(x, y):
    """out put : dgree, rho"""
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    degree = np.rad2deg(phi).reshape([-1,1])
    out = np. hstack((rho,degree))
    return out

def pol2cart(rho, degree):
    """in put : degree, rho"""
    phi = np.deg2rad(degree).reshape([-1,1])
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    out = np.hstack((x, y))
    return out

def circle_location(radius, ratio, degree, rotate_degree=0):
    """
    첫번째 원점에서 떨어져있는 값 radius 반지름길이임 (픽셀단위)
    예상각도 수정하고 싶으면 np.arrange 안에 각도 바꿔주면됨
    ratio : mm/pixel , output mm
    x , y 좌표 , rad, theta(degree)
   """
    degree = np.arange(-1*degree+rotate_degree, rotate_degree+degree+5 ,5)
    degree = degree.reshape([-1,1])
    rho = np.ones([degree.shape[0],1]) * radius
    xy = pol2cart(rho, degree).round()
    rtheta = np.hstack((degree, rho * ratio))
    out = np.hstack((xy, rtheta))
    return out

def find_zone(array):
    """
    :param array: 일반 배열받음
    :return: 주위로 둘러쌓인 0값을 제거하고 행렬 반환
    """
    y_axis = np.nonzero(array)[0]
    x_axis = np.nonzero(array)[1]
    y_min, y_max = np.min(y_axis), np.max(y_axis)
    x_min, x_max = np.min(x_axis), np.max(x_axis)
    result = array[y_min:y_max, x_min:x_max]
    return result

def u_f1(n, r, theta):
    shear_modulus = 26900 #unit:mpa
    poisson_ratio = 0.33
    theta = np.deg2rad(theta)
    plane_stress= (3 - poisson_ratio)/(1 + poisson_ratio)
    plane_strain = 3 - 4 * poisson_ratio

    result = r**(n/2) * (plane_stress + n/2 + (-1)**n * np.cos(n/2 * theta) - n/2*np.cos((n/2 - 2) * theta)) / (2 * shear_modulus)
    return result

def u_f2(n, r, theta):
    shear_modulus = 26900 #unit:mpa
    poisson_ratio = 0.33
    theta = np.deg2rad(theta)
    plane_stress= (3 - poisson_ratio)/(1 + poisson_ratio)
    plane_strain = 3 - 4 * poisson_ratio

    result = r**(n/2) * (plane_stress + n/2 -1*(-1)**n * np.sin(n/2*theta) - 0.5*n*np.sin((n/2-2)*theta)) / (2 * shear_modulus)
    return result

def v_f1(n, r, theta):
    shear_modulus = 26900 #unit:mpa
    poisson_ratio = 0.33
    theta = np.deg2rad(theta)
    plane_stress= (3 - poisson_ratio)/(1 + poisson_ratio)
    plane_strain = 3 - 4 * poisson_ratio

    result = r**(n/2) * (plane_stress - n/2 -(-1)**n * np.sin(n/2 * theta) + n/2*np.sin((n/2 -2)*theta)) / (2 * shear_modulus)
    return result

def v_f2(n, r, theta):
    shear_modulus = 26900 #unit:mpa
    poisson_ratio = 0.33
    theta = np.deg2rad(theta)
    plane_stress= (3 - poisson_ratio)/(1 + poisson_ratio)
    plane_strain = 3 - 4 * poisson_ratio

    result = r**(n/2) * (-1 * plane_stress + n/2 -1*(-1)**n * np.cos(n/2 *theta) - 0.5*n*np.cos((n/2 - 2)  * theta)) / (2 * shear_modulus)
    return result

def load_file(path, flip = 'None'):
    """ 폴더에 들어간 순서대로 이미지 추가 Dict형태 좌우 반전시 flip = 'flip' 사용"""
    files = glob.glob(path+'./*.jpg')
    if not files:
        print("Chck Phath")
        sys.exit()
    img = {}
    for i in files:
        im = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
        name = os.path.split(os.path.splitext(i)[0])[-1]
        if flip == 'flip':
            im = cv2.flip(im, 1)
        img.setdefault('%s'%name,im)
    return img


class G():
    def __init__(self, K1, K2, F_STRESS, r, theta):
        self.K1 = K1
        self.K2 = K2
        self.F_STRESS = F_STRESS
        self.r = r
        self.theta = np.deg2rad(theta)

    def g(self, effective_stress):
        result = 1/(2*np.pi*self.r) * ((self.K1*np.sin(self.theta)+ 2*self.K2*np.cos(self.theta))**2 +(self.K2*np.sin(self.theta))**2) \
                 + 2 * self.F_STRESS / (np.sqrt(2*np.pi*self.r))*np.sin(self.theta/2)*(self.K1*np.sin(self.theta)*(1+2*np.cos(self.theta)) + self.K2*(1+2*np.cos(self.theta)**2+np.cos(self.theta))) \
                 + self.F_STRESS**2 - 2*effective_stress**2
        return result

    def delta_k1(self):
        result = np.sin(self.theta)*(self.K1*np.sin(self.theta)+2*self.K2*np.cos(self.theta)) / (np.pi * self.r) \
                 + (np.sqrt(2)*self.F_STRESS*(2*np.cos(self.theta)+1)*np.sin(self.theta/2)*np.sin(self.theta))/np.sqrt(np.pi*self.r)
        return result

    def delta_k2(self):
        result = (2*self.K2*np.sin(self.theta)**2 + 4*(self.K1*np.sin(self.theta)+ 2*self.K2*np.cos(self.theta))*np.cos(self.theta))/(2*np.pi*self.r) \
                 + (np.sqrt(2)*self.F_STRESS*(2*np.cos(self.theta)**2 + np.cos(self.theta)+1)*np.sin(self.theta/2))/np.sqrt(np.pi*self.r)
        return result

    def delta_stress(self):
        result = 2*self.F_STRESS + ( np.sqrt(2)* np.sin(self.theta/2)* (self.K1 * (2*np.cos(self.theta)+1)*np.sin(self.theta) + self.K2*(2*np.cos(self.theta)**2 +np.cos(self.theta) +1)))/np.sqrt(np.pi*self.r)
        return result


"""
step1. make mask reference, current

"""

##이미지 로드
path = r'E:\experiment data\aluminum\2021-03-30\AL IMAGE\CTS_UVon_HSC\DIC'
make_mask(path)
img = load_file(path,flip='')


Run complete
Chck Phath


SystemExit: 

C:\Users\ghrms\anaconda3\envs\project\lib\site-packages\IPython\core\interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
"""
step2. set crack tip
"""
ratio = 0.0333

c_tip = Crack_tip(ratio, img.keys(), 15,3,32)

c_tip.append(175, 356,0) #Ref : 1512
c_tip.append(175, 356,0) #Cur : 2600
c_tip.append(175, 356,0) #Cur : 3578

c_tip.append(175, 356,0) #Peak : 4048
c_tip.append(175, 356,0) #after : 4049

c_tip.append(175, 356,0) # middle : 4163

c_tip.append(175, 356,0) #Peak : 4270
c_tip.append(175, 356,0) #after : 4271

c_tip.append(175, 356,0) #middle : 4333

c_tip.append(175, 356,0) #Peak : 4376
c_tip.append(175, 356,0) #after : 4377

c_tip.append(175, 356,0) #middle: 4453

c_tip.append(175, 356,0) #Peak : 4539
c_tip.append(175, 356,0) #after : 4540

c_tip.append(175, 356,0) #middle: 4631

c_tip.append(175, 356,0) #Peak : 4733
c_tip.append(175, 356,0) #after : 4734




c_tip.all_position()


In [ ]:
"""
step3.
#mat save code
exx = handles_ncorr.data_dic.strains.plot_exx_cur_formatted
eyy = handles_ncorr.data_dic.strains.plot_eyy_cur_formatted
exy = handles_ncorr.data_dic.strains.plot_exy_cur_formatted
uu = handles_ncorr.data_dic.displacements.plot_u_cur_formatted
vv = handles_ncorr.data_dic.displacements.plot_v_cur_formatted

save t_exx.mat exx
save t_exy.mat exy
save t_eyy.mat eyy
save t_uu.mat uu
save t_vv.mat vv

"""
file_num = ''

uu = io.loadmat(path+'/'+file_num+'/t_uu.mat')['uu']
vv = io.loadmat(path+'/'+file_num+'/t_vv.mat')['vv']
uu_resize = cv2.resize(uu,(0,0),fx=2, fy=2 )
vv_resize = cv2.resize(vv,(0,0),fx=2, fy=2 )

exx = io.loadmat(path+'/'+file_num+'/t_exx.mat')['exx']
eyy = io.loadmat(path+'/'+file_num+'/t_eyy.mat')['eyy']
exy = io.loadmat(path+'/'+file_num+'/t_exy.mat')['exy']
ee = fracture.effective_strain(exx, eyy, exy)
np.savetxt(path+'/'+file_num+'/ee.csv',ee)
exx_resize = cv2.resize(exx,(0,0),fx=2, fy=2 )
eyy_resize = cv2.resize(eyy,(0,0),fx=2, fy=2 )
exy_resize = cv2.resize(eyy,(0,0),fx=2, fy=2 )
ee_resize = cv2.resize(ee,(0,0),fx=2, fy=2 )
# cv2.flip(ee_resize,1,ee_resize)
# cv2.flip(uu_resize,1,uu_resize)
# cv2.flip(vv_resize,1,vv_resize)
msn_list = []

print(uu_resize[264,187],vv_resize[264,187])

plt.imshow(find_zone(ee_resize))
plt.contourf(find_zone(ee_resize), levels=np.arange(0,0.045,0.005))
plt.colorbar()
plt.savefig(path+'/'+file_num+'/g.png',dpi=300)

In [ ]:

"""
step4. select point
"""
radius = 1.5*25
radius_gap = 4
rotate = 0
cur = c_tip.all_position().loc[str('ff'+file_num)]

count = 0
radius_g = 0
for _ in range(20):
    if count == 0:
        location = circle_location(radius, ratio,90,rotate)
        radius_g += radius_gap
        count += 1
    else:
        location = np.vstack((location, circle_location(radius+radius_g, ratio,90,rotate)))
        radius_g += radius_gap




In [ ]:
plt.imshow(ee_resize, cmap='gray')

zone_cracktip_x = cur.x
zone_cracktip_y = cur.y

plt.scatter( zone_cracktip_x, zone_cracktip_y, s = 0.5,  c = 'red')
plt.scatter(location[:,0]+zone_cracktip_x,location[:,1]+zone_cracktip_y, s = 0.1,  c='red')
for N in range(2,2+1):
    h, b = [], []
    h_u, h_v = [], []
    b_u, b_v = [0,0], [0,0]
    b_u_extra = [0,0,0]
    b_v_extra = [0,0,0]
    b_u_loc = [0]*N*2
    b_v_loc = [0]*N*2

    for x, y, degree, rho in location:
        b_u1, b_v2, b_u2, b_v1 = [], [], [], []
        dis_u = uu_resize[int(y+zone_cracktip_y)][int(x +zone_cracktip_x)]
        dis_v = vv_resize[int(y+zone_cracktip_y)][int(x +zone_cracktip_x)]
        h_u = np.append(h_u, (dis_u)*0.001)
        h_v = np.append(h_v, (dis_v)*0.001)

        for i in range(N):
            n = i+1
            b_u1.append(u_f1(n, rho*0.001,degree))
            b_u2.append(u_f2(n, rho*0.001,degree) * -1)
            b_v1.append(v_f1(n, rho*0.001,degree))
            b_v2.append(v_f2(n, rho*0.001,degree) * -1)

        b_u_loc = np.vstack((b_u_loc,[np.hstack((b_u1, b_u2))]))
        b_v_loc = np.vstack((b_v_loc,[np.hstack((b_v1, b_v2))]))
        b_u_extra = np.vstack((b_u_extra,[1,0,-1 * y*ratio*0.001]))
        b_v_extra = np.vstack((b_v_extra,[0,1, 1 * x*ratio*0.001]))

    b_extra = np.vstack((b_u_extra[1:,],b_v_extra[1:,]))

    h= np.hstack((h_u,h_v))
    h = np.reshape(h,(-1,1))
    b = np.hstack((np.vstack((b_u_loc[1:,],b_v_loc[1:,])),b_extra))

    CD = np.dot( np.dot( np.linalg.inv( np.dot(np.transpose(b), b)) , np.transpose(b)) , h) * (2 * np.pi)**0.5
    msn = np.average((np.dot(b,CD)-h)**2)
    msn_list.append(msn)
    print('K1:', CD[0],'K2:', CD[N], 'Far:',CD[1] , N, msn)

In [ ]:
iteration = 500
youngs_modulus = 71700
poisson_ratio = 0.33

K1, K2, F_STRESS= 1,1,1
gk_list=[]

for i in range(iteration):
    A_matrix = [[0,0,0]]
    G_matrix = [[0]]
    for x, y, degree, rho in location:
        strain11 = exx_resize[int(y+zone_cracktip_y)][int(x +zone_cracktip_x)]
        strain22 = eyy_resize[int(y+zone_cracktip_y)][int(x +zone_cracktip_x)]
        strain12 = exy_resize[int(y+zone_cracktip_y)][int(x +zone_cracktip_x)]

        stress11 = youngs_modulus/(1-poisson_ratio**2)*(strain11+poisson_ratio*strain22)
        stress22 = youngs_modulus/(1-poisson_ratio**2)*(strain22+poisson_ratio*strain11)
        stress12 = youngs_modulus/(1-poisson_ratio**2)*((1-poisson_ratio)/2) * strain12
        sigma_effective = np.sqrt( ((stress11-stress22)**2 + stress22**2 + stress11**2 + 6*stress12**2) / 2)

        gk = G(K1,K2,F_STRESS,rho, degree)
        A_matrix = np.vstack((A_matrix,[gk.delta_k1(), gk.delta_k2(), gk.delta_stress()]))
        G_matrix = np.vstack((G_matrix,[gk.g(sigma_effective)]))

    A_matrix = -A_matrix[1:,]
    G_matrix = G_matrix[1:,]
    delta_matrix = np.dot(np.linalg.inv(np.dot(A_matrix.transpose(), A_matrix)),np.dot(A_matrix.transpose(),G_matrix)).flatten()
    gk_list.append(np.average(G_matrix))
    K1 = K1+delta_matrix[0]
    K2 = K2+delta_matrix[1]
    F_STRESS = F_STRESS + delta_matrix[2]


print(K1,K2, F_STRESS)
plt.plot(np.arange(iteration),gk_list)



In [ ]:
img.keys()

In [ ]:
"""종합파트"""
file_num = '4'
# cur = c_tip.all_position().loc[str('ff'+file_num)]
# zone_cracktip_x = cur.x
# zone_cracktip_y = cur.y

src1 = img['1'][:,100:]
src2 = img['3'][:,100:]
# pz = fracture.Plastic_zone(84,503)

dst = cv2.absdiff(src2, src1)
gas = cv2.GaussianBlur(dst,(0,0),10)
gas = np.where(gas>100,1, gas)

# von_deg = np.rad2deg(pz.von_mises()[1]).reshape([-1,1])
# von_rho = pz.von_mises()[0].reshape([-1,1])
# von_po = pol2cart(von_rho, von_deg)*1000*25
#
# tre_deg = np.rad2deg(pz.tresca()[1]).reshape([-1,1])
# tre_rho = pz.tresca()[0].reshape([-1,1])
# tre_po = pol2cart(tre_rho, tre_deg)*1000*25

plt.imshow(src2,cmap='gray')

plt.contourf(gas)

plt.colorbar()
# plt.contour(ee_resize, levels=15, colors='blue', linewidths=0.5)

# plt.scatter(von_po[:,0]+zone_cracktip_x,von_po[:,1]+zone_cracktip_y, s=0.01, c='yellow')
# plt.scatter(tre_po[:,0]+zone_cracktip_x,tre_po[:,1]+zone_cracktip_y, s=0.01, c='green')

plt.savefig(path+'/4049.png',dpi=300)
plt.show()

In [ ]:
np.max(gas)
